# Sorting

Sorting algorithms come up so frequently that they deserve their own section.

In general, we want to solve the problem:

> Given an array of unsorted values, how can we sort them in ascending order?


## 1. Bubble Sort

### 1.1. Algorithm
We "bubble up" the next highest unsorted number on each pass-through.

1. Start with 2 pointers pointing at the first two values in the array. 
2. Compare the left and right values. If `left_value > right_value`, swap them. Otherwise, do nothing.
3. Move both pointers one cell rightwards.
4. Repeat Steps 2-3 until we reach values that have already been sorted. This completes the first "pass-through" and means we have "bubbled up" the biggest number to the end of the array.
5. Start over. repeat Steps 1-4 to bubble up the second biggest number into the second to last position. Repeat this until we perform a pass through with no swaps.



In [76]:
def bubble_sort(array):
    """Bubble sort algorithm to sort an array into ascending order.

    Note we ignore edge cases for the sake of clarity.
    These are left as an exercise for the reader:
    - array is empty
    - array has only 1 element
    
    Parameters
    ----------
    array: list
        The array that we wish to sort.

    Returns
    -------
    array: list
        The input array sorted in-place.
    """
    # Initially the entire array is unsorted
    last_unsorted_index = len(array) - 1
    is_sorted = False

    while not is_sorted:
        # Set this to True before we pass through the elements, 
        # then if we need to perform a swap the array is not sorted so we set it to False
        is_sorted = True

        # Perform a pass-through
        for left_pointer in range(last_unsorted_index):
            right_pointer = left_pointer + 1

            if array[left_pointer] > array[right_pointer]:
                # Swap the values
                array[left_pointer], array[right_pointer] = array[right_pointer], array[left_pointer]
                is_sorted = False

        # The pass-through is finished so the next highest value has been "bubbled up".        
        last_unsorted_index -= 1

    return array

In [77]:
input_array = [8, 7, 5, 3, 1, 9, 0, 8, 23, 69, 420, 12]
bubble_sort(input_array)

[0, 1, 3, 5, 7, 8, 8, 9, 12, 23, 69, 420]

### 1.2. Complexity

Each pass through loops through one fewer element:

Pass-through number | Number of operations
---------|--------------
1  | N-1
2  | N-2
3  | N-3
4  | N-4
5  | N-5
...| ...
k  | N-k

So in total, there are $(N-1) + (N-2) + (N-3) + ... + 1$ comparisons. This is the sum of an arithmetic progression, which we can calculate as $\frac{N^2}{2}$.

Also worth noting that in the worst case - an input array in descending order - each comparison will also result in a swap. This does not affect the Big-O complexity but would slow down the run time in practice.
There are $\frac{N^2}{2}$ comparisons and up to $\frac{N^2}{2}$ swaps, resulting in $O(N^2)$ total operations.


The complexity is therefore **$O(N^2)$**.

In general, any nested loop should be a hint at quadratic time complexity.

## 2. Selection Sort

### 2.1. Algorithm

Find the smallest value from the unsorted part of the array and put it at the beginning.

1. Check each cell of the array from left to right to find the lowest value. Store the index of the running minimum value.
2. At the end of pass-through $j$ (starting at 0), swap the minimum value with the one at index $j$.
3. Repeat steps 1-2 until we reach a pass-through that would start at the end of the array, i.e. $j = N-1$

In [72]:
def selection_sort(array):
    """Selection sort algorithm to sort an array into ascending order.
    
    Parameters
    ----------
    array: list
        The array that we wish to sort.

    Returns
    -------
    array: list
        The input array sorted in-place.
    """
    array_length = len(array)

    # Loop through all array elements
    for pass_through_number in range(array_length):

        # Find the minimum element in the remaining unsorted array
        min_index = pass_through_number
        for idx_unsorted_section in range(pass_through_number + 1, array_length):
            if array[idx_unsorted_section] < array[min_index]:
                min_index = idx_unsorted_section

        # Swap the found minimum element with the first element
        array[pass_through_number], array[min_index] = array[min_index], array[pass_through_number]
    
    return array

In [74]:
input_array = [8, 7, 5, 3, 1, 9, 0, 8, 23, 69, 420, 12]
selection_sort(input_array)

[0, 1, 3, 5, 7, 8, 8, 9, 12, 23, 69, 420]

### 2.2. Complexity

As with bubble sort, on each pass-through we loop through one fewer element, so there are $\frac{N^2}{2}$ comparisons. But each pass-through only performs 1 swap, so the total number of operations is $\frac{N^2}{2} + N$.

This is still therefore **$O(N^2)$** complexity, but it should be about twice as fast as bubble sort.

## 3. Insertion Sort
### 3.1. Algorithm
Remove a value to create a gap, shift values along to move the gap leftwards, then fill the gap.

1. **Create a gap.** For the first pass-through, we temporarily remove the second cell (i.e. index 1) and store it as a temporary variable. This leaves a gap at that index.
2. **Shifting phase.** Take each value to the left of the gap and compare it to the temporary variable. if `left_val > temp_val`, move left value to the right. This has the same effect as moving the gap leftwards. As soon as we encounter a value where `left_val < temp_val` the shifting phase is complete.
3. **Fill the gap.** Insert the temporary value into the current gap.
4. **Repeat pass-throughs.** Steps 1-3 constitute a single pass-through. Repeat this until the pass through begins at the final index of the array. At this point the array is sorted.

In [82]:
def insertion_sort(array):
    """Insertion sort algorithm to sort an array into ascending order.
    
    Parameters
    ----------
    array: list
        The array that we wish to sort.

    Returns
    -------
    array: list
        The input array sorted in-place.
    """
    array_len = len(array)

    for pass_thru_number in range(1, array_len):
        # 1. Create a gap
        temp_val = array[pass_thru_number]
        gap_idx = pass_thru_number

        # 2. Shifting phase
        # Move leftwards from the gap and keep shifting elements right if they are greater than temp_val
        while (gap_idx > 0) and (array[gap_idx - 1] > temp_val):
            array[gap_idx] = array[gap_idx - 1]
            gap_idx -= 1

        # 3. Fill the gap
        array[gap_idx] = temp_val

    return array

In [83]:
input_array = [8, 7, 5, 3, 1, 9, 0, 8, 23, 69, 420, 12]
insertion_sort(input_array)

[0, 1, 3, 5, 7, 8, 8, 9, 12, 23, 69, 420]

### 3.2. Complexity

The worst case is when the input array is sorted in descending order. 

There are 4 types of operations that occur:


##### Comparisons
If we compare values to the left of the gap on each step, there will be 1 comparison on the first pass-through, 2 on the second, 3 on the third, etc.

So there are $1 + 2 + ... + N-1 = \frac{N^2}{2}$ comparisons in the worst case.

##### Shifts
Each comparison could result in a shift, so there are the same number of shifts as comparisons in the worst case.

##### Removals and Insertions
We remove 1 temp value per pass-through, so there are $N-1$ removals.

We insert that value at the end of each pass-through, so there are also $N-1$ insertions.


##### Total

Operation   | Number
------------|-------
Removals    | $N-1$
Comparisons | $\frac{N^2}{2}$
Shifts      | $\frac{N^2}{2}$
Insertions  | $N-1$

Overall there are $N^2 + 2N - 2$ operations, so the complexity is $O(N^2)$.